In [1]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import face_recognition_models
import dlib
import os
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates
from time import time
import matplotlib.pyplot as plt
import imutils
from imutils import face_utils
import face_recognition
import shutil

In [2]:
face_recognition_model = face_recognition_models.face_recognition_model_location()
face_encoder = dlib.face_recognition_model_v1(face_recognition_model)

In [3]:
landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]


In [4]:
mp_drawing = mp.solutions.drawing_utils

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1,min_detection_confidence=0.5)

### MP + Dlib utils


In [5]:
def landmark_pt_list(mesh_results,width,height):
    landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
    landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
    landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]

    if mesh_results.multi_face_landmarks:
        for i,face_landmarks in enumerate(mesh_results.multi_face_landmarks): 
            if i==0:
                raw_landmark_set = []
                for index in landmark_points_5_1:
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    landmark_point=dlib.point([x,y])
                    raw_landmark_set.append(landmark_point)
                    #display(landmark_point)
                all_points=dlib.points(raw_landmark_set)
#         return dlib.points([{
#             "nose_tip": [raw_landmark_set[0]],
#             "left_eye": raw_landmark_set[1:3],
#             "right_eye": raw_landmark_set[3:],
#         }])
        return all_points

def bounding_rect(detection_results,width,height):
    if detection_results.detections:
        for i,detection in enumerate(detection_results.detections):
            if i==0:
                # bbox data
                bbox = detection.location_data.relative_bounding_box
                xy_min = _normalized_to_pixel_coordinates(bbox.xmin, bbox.ymin, height,width)
                xy_max = _normalized_to_pixel_coordinates(bbox.xmin + bbox.width, bbox.ymin + bbox.height,height,width)
                if xy_min is None or xy_max is None:
                    #print("face out of frame")
                    return
                else:
                    xmin,ymin =xy_min
                    xmax,ymax = xy_max
                    #bbox_points = { "xmin" : xmin,"ymin" : ymin,"xmax" : xmax,"ymax" : ymax}
                    rectangle= dlib.rectangle(left=xmin, top=ymax, right=xmax, bottom=ymin)
                    return rectangle

def ret_encoding(filepath,num_jitters=1):
    #image_input = cv2.imread(filepath)
    #image_input = cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB)
    image_input =face_recognition.load_image_file(filepath)
    width,height=image_input.shape[:-1]
    detection_results = face_detection.process(image_input)
    mesh_results = face_mesh.process(image_input)
    
    all_points=  landmark_pt_list(mesh_results,width,height)  
    b_box=bounding_rect(detection_results,width,height)
    if (all_points is None) or (b_box is None):
        return 
    raw_landmark_set=dlib.full_object_detection(b_box,all_points)
    #display(all_points)
    #display(b_box)
    encodings=face_encoder.compute_face_descriptor(image_input, raw_landmark_set, num_jitters)

    return encodings

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Linear sorting

In [6]:
def get_img_list(folder):
    img_list=[]
    for file in os.listdir(folder):
        if not file.startswith('.') and os.path.isfile(os.path.join(folder, file)):
            filepath = os.path.join(folder, file)
            filepath=filepath.replace('\\' , '/')
            img_list.append(file)
    return img_list        
    print("got image list")

In [7]:
def get_d(enc1, enc2):
    enc1=np.array(enc1)
    enc2=np.array(enc2)
    d=np.linalg.norm(enc1 - enc2, axis=0)
    return d

In [8]:
# not df
def encode_list(folder, img_list):
    enc_dict={}
    for img in img_list:
        img_path = os.path.join(folder,img)
        encoding=(ret_encoding(img_path))
        enc_dict[img]=encoding 
    return enc_dict

In [9]:
# not df
def get_closest(folder, start_img, img_list, enc_dict):
    img_list.remove(start_img)
#     enc1=(ret_encoding(os.path.join(folder,start_img)))
    enc1=enc_dict[start_img]
    dist=[]
    dist_dict={}
    for img in img_list:
#         file2=filebase+str(i)+".jpg".replace('\\' , '/')
#         test_img = os.path.join(folder,img)
#         enc2=(ret_encoding(test_img))
        enc2 = enc_dict[img]
        if (enc1 is not None) and (enc2 is not None):
#             print(file2)
            d = get_d(enc1, enc2)
#             print(d)
            dist.append(d)
            dist_dict[d]=img
    dist.sort()
    print(len(dist))
    return dist[0], dist_dict[dist[0]]

In [10]:
def save_sorted(folder, image, counter, dist):
    sorted_name = "linear_sort_"+str(counter)+"_"+str(round(dist, 2))+".jpg"
    sortfolder="sorted2"
    newfolder = os.path.join(folder,sortfolder)
    old_name=os.path.join(folder,image)
    new_name=os.path.join(newfolder,sorted_name)
    if not os.path.exists(newfolder):
        os.makedirs(newfolder)
    shutil.copy(old_name, new_name)
    print('saved, ',sorted_name)


### dataframe creation and sorting

In [29]:
def encode_list_df(folder, img_list):
#     enc_dict={}
    csv_name="face_encodings.csv"
    col1="file_name"
    col2="encoding"
    curr=0
    total = len(img_list)

    # encodings column list for splitting
    col_list=[]
    for i in range(128):
        col_list.append(col2+str(i))

    #initializing the dataframe
    image_data=pd.DataFrame(columns=[col1, col2])

    
    for img in img_list:
        if curr%10==0:print(curr,"/",total)
        curr+=1
        filepath = os.path.join(folder,img)        
        filepath=filepath.replace('\\' , '/')  ## cv2 accepts files with "/" instead of "\"
        encodings=ret_encoding(filepath)
        print(encodings)
        if encodings is not None:              ## checking if a face is found
            data=pd.DataFrame({col1:img,col2:[np.array(encodings)]})
            image_data = pd.concat([image_data,data],ignore_index=True)  

    #splitting the encodings column
    output_data = pd.DataFrame(image_data[col2].to_list(), columns=col_list)
    #adding the filename column and then puting it first
    output_data[[col1]]=pd.DataFrame(image_data[col1].tolist(),index=image_data.index)
    clms = output_data.columns.tolist()
    clms = clms[-1:] + clms[:-1]
    output_data=output_data[clms]
    # saving without index
    output_data.to_csv(csv_name, index=False)
    df = pd.read_csv(csv_name)
    return output_data

In [34]:
def get_closest_df(folder, start_img, df_enc):
    if start_img == "median":
        enc1 = df_enc.median().to_list()
        print("in median")
        print(enc1)
        
    else:
#         enc1 = get 2-129 from df via stimg key
        enc1 = df_enc.loc[start_img].to_list()
        df_enc=df_enc.drop(start_img)
        print("in new img",len(df_enc.index))
        print(enc1)
    
#     img_list.remove(start_img)
#     enc1=enc_dict[start_img]
    
    dist=[]
    dist_dict={}
    for index, row in df_enc.iterrows():
#         print(row['c1'], row['c2'])
#     for img in img_list:
        enc2 = row
        if (enc1 is not None) and (enc2 is not None):
            d = get_d(enc1, enc2)
            dist.append(d)
            dist_dict[d]=index
    dist.sort()
#     print(len(dist))
    return dist[0], dist_dict[dist[0]], df_enc

In [35]:
# test if new and old make a face
def is_face(image):
    # For static images:
    # I think this list is not used
    IMAGE_FILES = []
    with mp_face_detection.FaceDetection(model_selection=1, 
                                        min_detection_confidence=0.6
                                        ) as face_detection:
        # image = cv2.imread(file)
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
#         detection_results = face_detection.process(image)

        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if not results.detections:
            is_face = False
        else:
            is_face = True
        # annotated_image = image.copy()
        # for detection in results.detections:
        #     is_face = True
        #     print('Nose tip:')
        #     print(mp_face_detection.get_key_point(
        #       detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
        #     mp_drawing.draw_detection(annotated_image, detection)
        # cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)

        return is_face


In [36]:
# test if new and old make a face
def test_pair(last_file, new_file):
    try:
        img = cv2.imread(new_file)
        height, width, layers = img.shape
        size = (width, height)
        print('loaded img 1')
        
        last_img = cv2.imread(new_file)
        last_height, last_width, last_layers = last_img.shape
        last_size = (last_width, last_height)
        print('loaded img 2')
        
        # test to see if this is actually an face, to get rid of blank ones/bad ones
        if is_face(img):
            print('new file is face')
            # if not the first image
#             if i>0:
            # blend this image with the last image
            blend = cv2.addWeighted(img, 0.5, last_img, 0.5, 0.0)
            print('blended faces')
            blended_face = is_face(blend)
            print('is_face ',blended_face)
            # if blended image has a detectable face, append the img
            if blended_face:
#                     img_array.append(img)
                print('is a face! adding it')
                return True
            else:
                print('skipping this one')
                return False
            # for the first one, just add the image
            # this may need to be refactored in case the first one is bad?
#             else:
#                 print('this is maybe the first round?')
#                 img_array.append(img)
        else:
            print('new_file is not face: ',new_file)
            return False

#         i+=1

    except:
        print('failed:',new_file)
        return False


In [37]:
folder="/Users/michaelmandiberg/Documents/projects-active/facemap_production/images1684374726.2699468"
img_list = get_img_list(folder)

# start_img = img_list[1]
start_img = "median"

# enc_dict = encode_list(folder, img_list)

df_enc = encode_list_df(folder, img_list)

df_enc.set_index('file_name', inplace=True)


#with lists/dicts
# dist=0
# for i in range(len(img_list)-1):
#     save_sorted(folder, start_img, i, dist)
#     dist, start_img = get_closest(folder, start_img,img_list, enc_dict)
    
#     print(dist)
#     print (start_img)
    
#     if dist > .37: 
#         continue

#with df
dist=0
print(len(df_enc.index))


    
for i in range(len(df_enc.index)-2):
    dist, start_img, df_enc = get_closest_df(folder, start_img,df_enc)
#     print(folder, start_img, i, dist)
    save_sorted(folder, start_img, i, dist)
    print(i)
    print(len(df_enc.index))
    print(dist)
    print (start_img)
    
    if dist > .37: 
        break
    
    
# I don't know why, but this isn't working
# for i in range(len(df_enc.index)-2):
#     dist, start_img, df_enc = get_closest_df(folder, start_img,df_enc)
# #     print(folder, start_img, i, dist)
#     save_sorted(folder, start_img, i, dist)

#     if i>0:
#         #test blend
# #         last_file = os.path.join(folder,)
#         blend_is_face = (test_pair(os.path.join(folder,last_img), os.path.join(folder,start_img)))
#         print('blend_is_face ',blend_is_face)
#         if blend_is_face:
# #         print(test_pair(last_img,start_img))
#             save_sorted(folder, start_img, i, dist)
#             last_img = start_img
# #         else:
# #             start_img = last_img
            
#     print(i)
#     print(len(df_enc.index))
#     print(dist)
#     print (start_img)
    
#     if dist > .37: 
#         break
        
       

    

0 / 335
-0.0964717
0.142492
0.0910655
-0.00349866
-0.0507545
-0.0103249
-0.102223
-0.0717896
0.170045
-0.0834678
0.229326
-0.00731818
-0.230333
-0.0314093
-0.0180307
0.144903
-0.139571
-0.106833
-0.0639788
-0.124009
0.0443067
0.128922
2.00123e-05
0.00976667
-0.109687
-0.325113
-0.0864109
-0.0817375
0.0900281
-0.12441
-0.0389369
0.0237422
-0.136374
-0.00466726
0.0316396
0.10178
-0.000603847
-0.0799527
0.190663
0.0267598
-0.162747
-0.0134973
0.0697842
0.327854
0.181498
-0.00219478
0.0374737
-0.0873678
0.107789
-0.218303
0.0557407
0.165346
0.107569
0.0942822
0.0681531
-0.209075
-0.00138056
0.116318
-0.138907
0.101014
0.137211
-0.0577639
-0.0391582
-0.052386
0.239204
0.0852815
-0.14812
-0.166974
0.0771841
-0.166524
-0.0413697
0.0944979
-0.137251
-0.162064
-0.281289
0.0777306
0.444678
0.11261
-0.166464
0.0491085
-0.0455827
-0.0220322
0.0849717
0.0982793
-0.123267
0.0149537
-0.104898
0.0555268
0.236652
0.00939588
-0.00140287
0.199718
-0.0191075
0.0526353
0.00881884
0.0673898
-0.0996038
-0.01

-0.0163912
0.0871951
0.0659046
-0.0150758
-0.0994404
0.0161839
0.0201268
-0.0297419
0.0988063
-0.0729173
0.202855
0.00122475
-0.237447
1.45286e-05
0.0268018
0.0781209
-0.0941311
-0.0813647
-0.160221
-0.0874623
0.0125079
0.0786709
0.00675448
-0.000935756
-0.0795832
-0.286864
-0.0884008
-0.0538905
0.117966
-0.114508
0.0291813
0.0186909
-0.124438
0.0636158
0.0275737
0.0540726
-0.0558722
-0.112662
0.234198
-0.0115062
-0.152421
-0.0292196
0.046354
0.258557
0.200542
-0.0252654
-0.00191496
-0.0368633
0.074247
-0.235575
0.0259242
0.1816
0.0981511
0.131408
0.0696398
-0.164821
0.032503
0.142963
-0.184972
0.0205996
0.0335193
-0.0969873
-0.0479382
-0.101804
0.200291
0.0976248
-0.142851
-0.17041
0.0703534
-0.231121
-0.036554
0.0727919
-0.134941
-0.135625
-0.232792
0.0598327
0.356928
0.212193
-0.1715
0.0077649
-0.0591736
-0.0211102
0.0631929
0.0491381
-0.120424
-0.0326462
-0.0984015
0.0666054
0.161345
-0.0515744
0.000452742
0.178587
-0.020251
0.0144181
-0.0286288
-0.0139433
-0.0921442
-0.00381897
-0

-0.0456935
0.169254
0.0904941
-0.0199934
-0.0884088
0.0491985
-0.0317014
-0.0514367
0.106383
-0.028341
0.18298
-0.0511539
-0.252101
-0.0533951
0.0119206
0.11856
-0.128906
-0.0920089
-0.142138
-0.0400672
-0.00656969
0.0760337
-0.00283997
-0.00304792
-0.0667597
-0.33124
-0.0510998
-0.0781965
0.123676
-0.106841
-0.00545605
-0.00476177
-0.121072
0.00728241
0.0380254
0.0600088
-0.0553277
-0.0705937
0.194702
0.0062591
-0.183868
-0.0134595
0.0195955
0.305205
0.210333
0.033787
0.0252087
-0.0462807
0.128026
-0.210433
0.0391677
0.170271
0.140128
0.0996859
0.0639552
-0.163259
0.0153925
0.15421
-0.211315
0.057116
0.105685
-0.0559563
-0.0551674
-0.0859853
0.21997
0.0715966
-0.171735
-0.17728
0.0906554
-0.191921
-0.0226141
0.0808063
-0.122703
-0.127065
-0.267323
0.0581686
0.372328
0.167315
-0.200853
-0.0600893
-0.036003
0.00729764
0.0630573
0.0973181
-0.134647
-0.0145922
-0.0694522
-0.00369093
0.153265
-0.0266071
-0.00432491
0.200026
0.00495829
0.00540759
-0.0490823
-0.00738861
-0.08365
-0.00220896


-0.0478135
0.114858
0.0915099
0.0114199
-0.0374919
-0.0600429
-0.0448893
-0.0885367
0.101607
-0.0472068
0.20389
-0.0428411
-0.241988
-0.0692414
-0.0702733
0.132953
-0.131864
-0.0647426
-0.110821
-0.106193
0.0602283
0.0688204
0.00320613
-0.0437913
-0.0893039
-0.282027
-0.110112
-0.0778884
0.0804577
-0.0596226
0.0419224
0.0443273
-0.163657
-0.0348709
0.0163031
0.0385652
-0.0143312
-0.0529142
0.166254
0.0154447
-0.102624
-0.0204416
0.0250485
0.285572
0.236913
0.0506551
0.023622
-0.0437888
0.0557023
-0.290971
0.02382
0.142514
0.074405
0.0900161
0.0795921
-0.163596
0.0113219
0.128302
-0.149162
0.0648149
0.0537647
-0.0604728
-0.0673839
-0.0317979
0.198555
0.0685469
-0.132418
-0.100477
0.160997
-0.184521
-0.0186863
0.0265692
-0.107711
-0.136802
-0.297257
0.066098
0.467764
0.155367
-0.172359
0.024368
-0.0753397
0.0228155
0.0885845
0.000475958
-0.115766
-0.0363915
-0.103198
0.0267693
0.175173
-0.04955
-0.0277115
0.218915
-0.0426938
0.0227472
-0.0016195
-0.00974925
-0.0512797
-0.0120213
-0.08268

-0.0280824
0.142198
0.0913398
-0.0245803
-0.0683145
-0.00537311
-0.0373903
-0.0273323
0.100652
-0.038528
0.19169
-0.0554613
-0.224614
0.00654549
-0.0297357
0.0865228
-0.135707
-0.0571515
-0.127343
-0.0591239
0.0856687
0.0809903
-0.022933
0.0207385
-0.0646907
-0.297969
-0.0931132
-0.0731255
0.0903565
-0.094477
-0.00592655
0.0108424
-0.105375
0.00898153
0.0246856
0.0493516
-0.0419721
-0.0942576
0.133701
-0.0108868
-0.168496
-0.045648
0.0322556
0.274863
0.209022
0.0498362
0.00859797
-0.0711218
0.0583904
-0.222642
0.0110944
0.199117
0.0421873
0.118329
0.0850527
-0.148027
0.0324737
0.109868
-0.131537
0.0306102
0.0692779
-0.073357
-0.0331587
-0.100938
0.168468
0.0640245
-0.141631
-0.163833
0.0651048
-0.215663
-0.0767935
0.0731939
-0.118629
-0.158981
-0.243594
0.0294221
0.412467
0.196638
-0.186672
0.0160366
-0.0575863
-0.00976362
0.14707
0.0854017
-0.0905576
0.015734
-0.121624
0.0388208
0.172032
-0.0732945
-0.000777179
0.19161
-0.0348234
0.0240804
-0.0142702
0.0194634
-0.0852994
0.0415363
-0.

...

saved,  linear_sort_116_0.26.jpg
116
218
0.2568605542415836
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_5_1170308721.jpg
in new img 217
[-0.04468922317028046, 0.11854317784309387, 0.06085618957877159, -0.006442000158131123, -0.06499717384576797, 0.024661004543304443, -0.04021918401122093, -0.035947926342487335, 0.11692120879888535, -0.027850128710269928, 0.14843817055225372, -0.021397609263658524, -0.2326188087463379, 0.012406878173351288, -0.010098516941070557, 0.08476907014846802, -0.11477604508399963, -0.08642563968896866, -0.1901872754096985, -0.11864527314901352, 0.034395553171634674, 0.0863310694694519, -0.013600137084722519, 6.240606307983398e-05, -0.14692208170890808, -0.2922288477420807, -0.0702550858259201, -0.07739968597888947, 0.08530980348587036, -0.10950739681720734, 0.03533536195755005, -0.005068860948085785, -0.1407698392868042, -0.0019346997141838074, 0.06339196860790253, 0.0009881779551506042, -0.03147806227207184, -0.06777975708246231, 0.18276268243789

saved,  linear_sort_135_0.29.jpg
135
199
0.29353785397739474
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_83_1396146747.jpg
in new img 198
[-0.05124577879905701, 0.13680200278759003, 0.0533570870757103, -0.07955162227153778, -0.11929228901863098, 0.07253049314022064, 0.013926506042480469, -0.018688876181840897, 0.07877469062805176, -0.026661328971385956, 0.1690485030412674, -0.02755967155098915, -0.25632020831108093, -0.0009207502007484436, 0.0007870420813560486, 0.08810822665691376, -0.107535220682621, -0.1011306494474411, -0.17588798701763153, -0.08302195370197296, 0.041998837143182755, 0.04991549998521805, -0.04566618800163269, 0.018896296620368958, -0.06258262693881989, -0.2831321358680725, -0.04262499883770943, -0.023870036005973816, 0.12002135813236237, -0.10056567192077637, 0.019710971042513847, -0.03310540318489075, -0.15374107658863068, 0.023335173726081848, 0.03370893746614456, 0.04589313268661499, -0.09400481730699539, -0.099098339676857, 0.18958649039268494, -

saved,  linear_sort_156_0.29.jpg
156
178
0.28804135428652217
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_72_1288959619.jpg
in new img 177
[-0.007695414125919342, 0.09788956493139267, 0.08701357245445251, -0.07918476313352585, -0.0614243820309639, -0.04848509281873703, -0.015739720314741135, -0.05115547031164169, 0.15575501322746277, -0.05262477695941925, 0.16673915088176727, -0.04668816179037094, -0.23668301105499268, -0.00042369216680526733, -0.0128556489944458, 0.10311172902584076, -0.12690433859825134, -0.0955619215965271, -0.14626236259937286, -0.12679839134216309, 0.05068245157599449, 0.09383723884820938, 0.012201175093650818, 0.027199797332286835, -0.07239585369825363, -0.28208646178245544, -0.08501532673835754, -0.06888747215270996, 0.11293769627809525, -0.09151396155357361, 0.005526278167963028, 0.039185263216495514, -0.13121500611305237, -0.056070052087306976, 0.032060086727142334, 0.02717815339565277, -0.09918700903654099, -0.1471385657787323, 0.143982589244842

saved,  linear_sort_179_0.35.jpg
179
155
0.3466952758676927
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_119_771534341.jpg
in new img 154
[-0.04580157995223999, 0.07475516945123672, 0.051733437925577164, -0.04303819313645363, -0.1320878565311432, -0.0029244348406791687, -0.012709133327007294, -0.01504119485616684, 0.11697517335414886, -0.02942012995481491, 0.1730901449918747, -0.048340700566768646, -0.24202248454093933, 0.020728982985019684, -0.005036279559135437, 0.09969740360975266, -0.16334998607635498, -0.04157423973083496, -0.14011721312999725, -0.08588963747024536, 0.05469405651092529, 0.10745114833116531, -0.06000925600528717, 0.013188086450099945, -0.07332734018564224, -0.2719162404537201, -0.06302006542682648, -0.09239508956670761, 0.08180096000432968, -0.10826493799686432, 0.031261831521987915, 0.0008291900157928467, -0.14132575690746307, -0.04207716882228851, 0.028161190450191498, -0.006037425249814987, -0.05436653643846512, -0.09852482378482819, 0.148288875818

saved,  linear_sort_205_0.33.jpg
205
129
0.3287548050132244
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_245_1291582813.jpg
in new img 128
[-0.05348806083202362, 0.12244526296854019, 0.04834645986557007, -0.04338374733924866, -0.08198550343513489, -0.038951046764850616, -0.06404609978199005, -0.04544663056731224, 0.11368613690137863, -0.009696956723928452, 0.1332332193851471, -0.04564755782485008, -0.2510198950767517, 0.0028372928500175476, -0.043719369918107986, 0.04971704259514809, -0.1290586143732071, -0.10603147745132446, -0.12923553586006165, -0.1387133002281189, 0.09379982203245163, 0.10196348279714584, -0.03913825750350952, 0.045140981674194336, -0.07953950762748718, -0.24146801233291626, -0.04830401390790939, -0.09077134728431702, 0.08971109241247177, -0.15343967080116272, 0.03968701511621475, 0.022463075816631317, -0.10753411799669266, -0.04216908663511276, 0.0005399361252784729, 0.017730053514242172, -0.06066007539629936, -0.12928514182567596, 0.1594868451356887